In [ ]:
## Main function for signal processing and feature extraction activation
def featuer_extraction(input_root,file_name, start, end, label):
    data = load_file(f"{input_root}/{file_name}")
    #Cut relevant part using end,start
    data = data.truncate(before=start-1, after=end-1)
    ### Apply Median filter with window=5 to reduce white noise (normal noise)
    TA_med_filter= pd.DataFrame({'Ax':medfilt(data.Ax,kernel_size= 3),
                                   'Ay':medfilt(data.Ay,kernel_size=3),
                                   'Az':medfilt(data.Az,kernel_size=3)})
    
    #plot_acc_together(data, TA_med_filter, 'Compare TA raw and with median Filter')
    ### A third order low-pass Butterworth filter with a cutoff frequency of 20 Hz.
    fs_in = 50       # sample rate of source data [Hz]
    cutoff = 20 # The Low-pass filter, passes signals with a frequency lower than a certain cutoff frequency and attenuates signals with frequencies higher than the cutoff frequency.
    order=3
    fs_out = 20
    TA_lowpass_filter = pd.DataFrame({'Ax':butter_lowpass_filter(TA_med_filter.Ax, cutoff, fs_in, order,fs_out),
                                   'Ay':butter_lowpass_filter(TA_med_filter.Ay, cutoff, fs_in, order,fs_out),
                                   'Az':butter_lowpass_filter(TA_med_filter.Az, cutoff, fs_in, order,fs_out)})
    #plot_acc_together(TA_med_filter, TA_lowpass_filter, 'Compare TA with median Filter and low pass filter')
    #### Gravity calculation
    fs= 50       # sample rate of source data [Hz]
    cutoff_gravity = 0.25 # The Low-pass filter, passes signals with a frequency lower than a 
                            #certain cutoff frequency and attenuates signals with frequencies higher than the cutoff frequency.
    order=3
    rp = 0.01 #passband ripple
    rs = 100 # stop band 
    #The Gravity Acceleration (GA) component is taken directly from the result of applying the LPF to the TA_lowpass_filter 
    Gravity = pd.DataFrame({'Ax':IIR_lowpass_filter(TA_lowpass_filter.Ax, cutoff_gravity, fs, rp, rs, order),
                               'Ay':IIR_lowpass_filter(TA_lowpass_filter.Ay, cutoff_gravity, fs, rp, rs, order),
                               'Az':IIR_lowpass_filter(TA_lowpass_filter.Az, cutoff_gravity, fs, rp, rs, order)})
    # Body Acceleraton (BA) component is taken as the difference between the original signal and the GA component
    
    BA_data= pd.DataFrame({'Ax':TA_lowpass_filter.Ax-Gravity.Ax,
                               'Ay':TA_lowpass_filter.Ay-Gravity.Ay,
                               'Az':TA_lowpass_filter.Az-Gravity.Az})
    
    #plot_acc_together(data_lowpass_filter, body_acceleration, 'Compare TA with BA filter')
    ## calculate jerk from BA
    dt = 1/50
    jerk_BA = jerk_calc(BA_data,dt)
    # Sloding window - each with a span of 2.56 sec and an overlap of 50% . This was determined according to 
    # values in literature, and is known to cpature human movment. 128 sample = 2.56 sec * 5-Htz sampling rate
    # overlaping - 64 samles
    window_size = 128
    features = pd.DataFrame()
    # TA = Total acceleration
    features = time_freq_domain_feature_calc('TA',TA_lowpass_filter,window_size,features)
    # BA = body acceleration
    features = time_freq_domain_feature_calc('BA',BA_data,window_size,features)
    # GA = Gravity acceleration
    features = time_freq_domain_feature_calc('GA',Gravity,window_size,features)
    # BJ = Body Jerk
    features = time_freq_domain_feature_calc('BJ',jerk_BA,window_size,features)  
    ### Compute cross correlation
    features = cross_correlation(features, BA_data, window_size)
    ### selecting only sliding window with 50% overlapping
    features = features[128::64]
    ### setting the labling of this set
    features['label']= np.ones(len(features))*label
    return features 